<h1 style="text-align: center;">Most Active Stocks: Data Preparation and EDA</h1>

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install polygon-api-client

Note: you may need to restart the kernel to use updated packages.


#### Web scrapping - Yahoo Finance Most active stock - Date: 02/20/2025 

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from datetime import datetime
import time

# Set up Google Chrome WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Initialize the stock data list
stock_data = []

# Number of pages to scrape
num_pages = 10

# Scrape data from each page by updating the URL with `start` parameter
for page in range(num_pages):
    print(f"Scraping page {page+1}")
    
    # Construct URL with the correct start and count for pagination
    start = page * 25
    url = f'https://finance.yahoo.com/markets/stocks/most-active/?start={start}&count=25'
    
    # Open the page
    driver.get(url)
    
    # Wait for the table to load
    wait = WebDriverWait(driver, 90)
    table = wait.until(EC.presence_of_element_located((By.XPATH, "//table[1]")))

    # Extract table rows (skip the header)
    rows = table.find_elements(By.TAG_NAME, 'tr')

    # Iterate over the rows and extract data
    for row in rows[1:]:  # Skip header row
        cols = row.find_elements(By.TAG_NAME, 'td')
        if len(cols) > 0:
            symbol = cols[0].text.strip()
            name = cols[1].text.strip()
            
            # Extract price using a more specific XPath
            try:
                price = row.find_element(By.XPATH, './/td[3]//span').text.strip()  # XPath for the price
            except:
                price = None  # Handle case where price is missing
            
            change = cols[3].text.strip()
            percent_change = cols[4].text.strip()
            volume = cols[5].text.strip()
            market_cap = cols[6].text.strip()
            
            stock_data.append([symbol, name, price, change, percent_change, volume, market_cap])
    
    # Pause for a few seconds to avoid hitting the server too quickly
    time.sleep(2)

# Close the browser after scraping all pages
driver.quit()

# Convert the list to a DataFrame
df = pd.DataFrame(stock_data, columns=['Symbol', 'Name', 'Price', 'Change', 'Percent Change', 'Volume', 'Market Cap'])

# Display the DataFrame
print(df)

# Get the current date and time in a format suitable for filenames
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Create the filename with the date and time included
file_name = f"most_active_stocks_{current_time}.xlsx"

# Export the DataFrame to Excel with the new filename
df.to_excel(file_name, index=False)


Scraping page 1
Scraping page 2
Scraping page 3
Scraping page 4
Scraping page 5
Scraping page 6
Scraping page 7
Scraping page 8
Scraping page 9
Scraping page 10
    Symbol                             Name Price  Change Percent Change  \
0     PLTR       Palantir Technologies Inc.        106.27          -5.79   
1     SMCI       Super Micro Computer, Inc.         59.27          -0.98   
2     INTC                Intel Corporation         26.09          +0.37   
3     NVDA               NVIDIA Corporation        140.11          +0.88   
4     BABA    Alibaba Group Holding Limited        135.97         +10.18   
..     ...                              ...   ...     ...            ...   
245    ADI             Analog Devices, Inc.        243.82          +2.16   
246   TCOM           Trip.com Group Limited         66.11          -0.57   
247   PTEN       Patterson-UTI Energy, Inc.          8.83          -0.03   
248   VTRS                     Viatris Inc.         11.06          +0.13   
249

#### To check 3-day uptrend for price and volume - API call for Polygon.io

In [6]:
from polygon import RESTClient
import pandas as pd
import time
from datetime import datetime, timedelta

# Function to check 3-day uptrend for price and volume using Polygon.io
def check_3_day_uptrend(df, api_key):
    time.sleep(1)  # Add delay to avoid rate limits
    
    # Initialize Polygon RESTClient
    client = RESTClient(api_key)
    
    # Get the current date and calculate the date range for the last 4 days
    to_date = datetime.now()
    from_date = to_date - timedelta(days=4)  # Get the last 4 days of data
    
    # Convert dates to strings in 'YYYY-MM-DD' format
    from_date_str = from_date.strftime('%Y-%m-%d')
    to_date_str = to_date.strftime('%Y-%m-%d')
    
    # Create new columns for price_up and volume_up
    df['price_up'] = "NO"
    df['volume_up'] = "NO"
    
    # Loop through each stock symbol in the DataFrame
    for idx, row in df.iterrows():
        stock_symbol = row['Symbol']
        
        try:
            # Fetch the last 4 days of data (OHLCV) using get_aggs method
            aggs = client.get_aggs(stock_symbol, 1, 'day', from_=from_date_str, to=to_date_str)
        except Exception as e:
            print(f"Error fetching data for {stock_symbol}: {e}")
            continue
        
        # Ensure we have at least 3 days of data
        if len(aggs) < 3:
            continue  # Skip this stock symbol if there is not enough data
        
        # Prepare the data (Price and Volume)
        close_prices = [agg.close for agg in aggs]  # Closing prices
        volumes = [agg.volume for agg in aggs]  # Volumes
        
        # Check for price uptrend (3-day)
        if close_prices[0] < close_prices[1] and close_prices[1] < close_prices[2]:
            df.at[idx, 'price_up'] = "YES"
        
        # Check for volume uptrend (3-day)
        if volumes[0] < volumes[1] and volumes[1] < volumes[2]:
            df.at[idx, 'volume_up'] = "YES"
        
        # Exclude stocks that are down today (compare last two closing prices)
        if close_prices[-1] < close_prices[-2]:  # Compare the most recent close to the previous one
            df.at[idx, 'price_up'] = "NO"
            df.at[idx, 'volume_up'] = "NO"
    
    return df



In [7]:
# API Key from Polygon.io
api_key = 'IiVN4gR_7dpnNQ0H8J7N304wrr_379QZ'

# df is already loaded with stock symbols, run the check_3_day_uptrend function
df = check_3_day_uptrend(df, api_key)

# Create the filename with the date and time included
file_name = f"most_active_stocks-3dayUP{current_time}.xlsx"

# Export the DataFrame to Excel with the new filename
df.to_excel(file_name, index=False)

# Display the DataFrame with the new columns
print(df)

    Symbol                             Name Price  Change Percent Change  \
0     PLTR       Palantir Technologies Inc.        106.27          -5.79   
1     SMCI       Super Micro Computer, Inc.         59.27          -0.98   
2     INTC                Intel Corporation         26.09          +0.37   
3     NVDA               NVIDIA Corporation        140.11          +0.88   
4     BABA    Alibaba Group Holding Limited        135.97         +10.18   
..     ...                              ...   ...     ...            ...   
245    ADI             Analog Devices, Inc.        243.82          +2.16   
246   TCOM           Trip.com Group Limited         66.11          -0.57   
247   PTEN       Patterson-UTI Energy, Inc.          8.83          -0.03   
248   VTRS                     Viatris Inc.         11.06          +0.13   
249     KC  Kingsoft Cloud Holdings Limited         20.17          +0.84   

     Volume Market Cap price_up volume_up  
0    -5.17%   240.421M       NO        NO  